_THIS NOTEBOOK IS NOT CURRENTLY WORKING (due to issues with the AWS dtata)_

# Using data from AWS

A significant amount of Earth System Model (ESM) data is publicly available online, including data from the CESM Large Ensemble, CMIP5, and CMIP6 datasets. For accessing a single file, we can specify the file (typically netcdf or zarr format) and its location and then use fsspec (the "Filesystem Spec+  python package) and xarray  to create a array.dataset.  For several files, the intake_esm python module (https://github.com/intake/intake-esm) is particularly nice for obtaining the data and put it into an xarray.dataset.

This notebook assumes familiarity with the Tutorial Notebook.  It additionally shows how to gather data from an ESM collection, put it into a dataset, and then create simple plots using the data with ldcpy.

#### Example Data

The example data we use is from the CESM Large Ensemble, member 31. This ensemble data has been lossily compressed and reconstructed as part of a blind evaluation study of lossy data compression in LENS (e.g., http://www.cesm.ucar.edu/projects/community-projects/LENS/projects/lossy-data-compression.html or https://gmd.copernicus.org/articles/9/4381/2016/).

Most of the data from the CESM Large Ensemble Project has been made available on Amazon Web Services (Amazon S3), see http://ncar-aws-www.s3-website-us-west-2.amazonaws.com/CESM_LENS_on_AWS.htm .

For comparison purposes, the original (non-compressed) data for Ensemble 31 has recently been made available on Amazon Web Services (Amazon S3)  in the "ncar-cesm-lens-baker-lossy-compression-test" bucket.

In [ ]:
# Add ldcpy root to system path
import sys
sys.path.insert(0,'../../../')

# Import ldcpy package
# Autoreloads package everytime the package is called, so changes to code will be reflected in the notebook if the above sys.path.insert(...) line is uncommented.
%load_ext autoreload
%autoreload 2
import ldcpy

import intake
import fsspec
import xarray as xr

#display the plots in this notebook
%matplotlib inline

## Method 1: using fsspec and xr.open_zarr

First, specify the filesystem and location of the data.  Here we are accessing the original data from CESM-LENS ensemble 31, which is available on Amazon S3 in the store named _"ncar-cesm-lens-baker-lossy-compression-test"_ bucket.

First we listing all available files (which are timeseries files containing a single variable) for that dataset. Note that unlike in the TutorialNotebook (which used NetCDF files), these files are all zarr format. Both monthly and daily data is available.

In [ ]:
fs = fsspec.filesystem('s3', anon=True)
stores = fs.ls("ncar-cesm-lens-baker-lossy-compression-test/lens-ens31/")[1:]
stores[:]

Then we select the file from the store that we want and open it as an xarray.Dataset using xr.open_zarr(). Here we grab data for the first 2D daily variable, FLNS (net longwave flux at surface, in $W/m^2$), in the list (accessed by it location -- stores[0]).

In [ ]:
store = fs.get_mapper(stores[0])
ds_flns = xr.open_zarr(store, consolidated=True, decode_times=False)
ds_flns

The above returned an xarray.Dataset.

Now let's grab the TMQ (Total vertically integrated precipitatable water) and the TS (surface temperature data) and PRECT (precipitation rate) data from AWS.

In [ ]:
#TMQ data
store2 = fs.get_mapper(stores[16])
ds_tmq = xr.open_zarr(store2, consolidated=True, decode_times=False)
#TS data
store3 = fs.get_mapper(stores[20])
ds_ts = xr.open_zarr(store3, consolidated=True, decode_times=False)
#PRECT data
store4 = fs.get_mapper(stores[11])
ds_prect = xr.open_zarr(store4, consolidated=True, decode_times=False)

Now we have the original data for FLNS and TMQ and TS and PRECT. Next we want to get the lossy compressed variants to compare with these. 

## Method 2: Using intake_esm

Now we will demonstrate using the intake_esm module to get the lossy variants of the files retrieved above.  We can use the intake_esm module to search for and open several files as xarray.Dataset objects. The code below is modified from the intake_esm documentation, available here: https://intake-esm.readthedocs.io/en/latest/?badge=latest#overview.

We want to use ensemble 31 data from the CESM-LENS collection on AWS, which (as explained above) has been subjected to lossy compression. Many catalogs for publicly available datasets are accessible via intake-esm can be found at https://github.com/NCAR/intake-esm-datastore/tree/master/catalogs, including for CESM-LENS. We can open that collection as follows (see here: https://github.com/NCAR/esm-collection-spec/blob/master/collection-spec/collection-spec.md#attribute-object): 

In [ ]:
aws_loc = "https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json"
aws_col = intake.open_esm_datastore(aws_loc)
aws_col

Next, we search for the subset of the collection (dataset and variables) that we are interested in.  Let's grab FLNS, TMQ, and TS daily data from the atm component for our comparison (available data in this collection is listed here: http://ncar-aws-www.s3-website-us-west-2.amazonaws.com/CESM_LENS_on_AWS.htm).

In [ ]:
#we want daily data for FLNS, TMQ, and TS
aws_col_subset = aws_col.search(component='atm', frequency='daily', experiment='20C',
                        variable=["FLNS", "TS", "TMQ"])
#display header info to verify that we got the right variables
aws_col_subset.df.head()

Then we load matching catalog entries into xarray datasets (https://intake-esm.readthedocs.io/en/latest/api.html#intake_esm.core.esm_datastore.to_dataset_dict).  We create a dictionary of datasets:

In [ ]:
dset_dict = aws_col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True, "decode_times": True}, 
                                   storage_options={"anon": True}, cdf_kwargs={'chunks': {}, 'decode_times': False})
dset_dict

Check the dataset keys to ensure that what we want is present.  Here we only have oneentry in the dictonary as we requested the same time period and output frequency for all variables:

In [ ]:
dset_dict.keys()

Finally, put the dataset that we are interested from the dictionary into its own dataset variable. (We want the 20th century daily data -- which is our only option.) 

Also note from above that there are 40 ensemble members - and we just want ensemble 31 (member_id = 30 as can be seen in the coordinates above).

In [ ]:
aws_ds=(dset_dict['atm.20C.daily'])
aws_ds = aws_ds.isel(member_id=30)
aws_ds

Now we have datasets for the original and the lossy compressed data for FLNS, TMQ, and TS, which we can extract into a dataset for each variable:

In [ ]:
#extract the three variables from aws_ds as datasets
aws_flns = aws_ds['FLNS'].to_dataset()
aws_tmq = aws_ds['TMQ'].to_dataset()
aws_ts = aws_ds['TS'].to_dataset()
aws_ts

## why can't I get PRECT?

In [ ]:
#we want daily data for PRECT
aws_col_subset2 = aws_col.search(component='atm', frequency='daily', experiment='20C',
                        variable=["PRECT"])
aws_col_subset = aws_col.search(component='atm', frequency='daily', experiment='20C',
                        variable=["FLNS", "TS", "TMQ"])
#display header info to verify that we got the right variables
aws_col_subset2.df.head()

In [ ]:
dset_dict = aws_col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True, "decode_times": True}, 
                                   storage_options={"anon": True})
dset_dict

In [ ]:
dset_dict2 = aws_col_subset2.to_dataset_dict(zarr_kwargs={"consolidated":True, "decode_times": True}, 
                                   storage_options={"anon": True})

dset_dict2

# Use _ldcpy_ to compare the original data to the lossy compressed data

To use ldcpy, we need to group the data that we want to compare (like variables) into dataset collections.  In the Tutorial Notebook, we used ldcpy.open_datasets() to do this as we needed to get the data from the NetCDF files.  Here we already loaded the data from AWS into datasets, so we just need to use ldcpy.collect_datasets() to form collections of the datasets that we want to compare.

ldcpy.collect_datasets() requires the following three arguments:

- _varnames_ : the variable(s) of interest to combine across files (typically the timeseries file variable name)
- _list_of_ds_ : a list of the xarray datasets
- _labels_ : a corresponding list of names (or labels) for each dataset in the collection

Note: This function is a wrapper for xarray.concat(), and any additional key/value pairs passed in as a dictionary are used as arguments to xarray.concat(). 

We will create 3 collections for ldcpy (one each for FLNS, TMQ, and TS) and assign labels "original" and "lossy" to the respective datasets.

In [ ]:
#FLNS collection
col_flns = ldcpy.collect_datasets(["FLNS"], [ds_flns, aws_flns], ['original', 'lossy'])

In [ ]:
col_flns

**Why is the time slice number so large?  IT is starting in 1850 instead of 1920???***

In [ ]:
#TMQ collection
col_tmq = ldcpy.collect_datasets(["TMQ"], [ds_tmq, aws_tmq], ['original', 'lossy'])

In [ ]:
#Ts collection
col_ts = ldcpy.collect_datasets(["TS"], [ds_ts, aws_ts], ['original', 'lossy'])

Now that we have our collections, let's make some plots!

In [ ]:
#comparison between mean FLNS values in col_flns "original" and "lossy" datasets
ldcpy.plot(col_flns, "FLNS", sets=['original', 'lossy'], metric='mean', calc_ssim = True)

In [ ]:
# plot of mean winter TS values in the col_ts "original" and "lossy" datasets
ldcpy.plot(col_ds, "TS", sets=['original','lossy'], metric='mean', subset="winter")

## Make your own catalog for intake-esm


The original data for FLNS and TMQ and TS and PRECT (from Amazon S3 in the "ncar-cesm-lens-baker-lossy-compression-test" bucket)  was loaded above using method 1.  An alternative would be to create our own catalog for this data for use with intake-esm.  To illustrate this, we created a test_catalog.csv and test_collection.json file for this particular simple example.

We first open our collection.

In [ ]:
my_col_loc = "./collections/test_collection.json"
my_col = intake.open_esm_datastore(my_col_loc)
my_col

In [ ]:
#printing the head() gives us the file names
my_col.df.head()

Let's load all of these into our dictionary! (So we don't need to do the search to make a subset of variables as above in Method 2.)

In [ ]:
my_dset_dict = my_col.to_dataset_dict(zarr_kwargs={"consolidated": True, "decode_times": True}, 
                                   storage_options={"anon": True})
my_dset_dict

In [ ]:
#we again just want the 20th century daily data
my_ds=(my_dset_dict['atm.20C.daily'])
my_ds

Now we can make a dataset for each variable as before.

In [ ]:
my_ts = my_ds["TS"].to_dataset()
my_tmq = my_ds["TMQ"].to_dataset()
my_prect = my_ds["PRECT"].to_dataset()
my_flns = my_ds["FLNS"].to_dataset()

And now we can form new collections and do comparisons.